# Prediction of restaurant ratings: Data Cleaning Notebook 

In this part we are cleaning the data and extract new features from existing one.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize
import re
import os
import ast
from sklearn import preprocessing

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
df = pd.read_csv("../data/data.csv", 
                          encoding='utf_8', 
                          dtype = 'unicode',
                          parse_dates = True,
                          infer_datetime_format = True,
                          low_memory=False)
df = df.drop("Unnamed: 0", axis = 1)


In [4]:
categ = pd.read_csv("../data/categories.txt")
categ.Cuisine = categ.Cuisine.str.rstrip()
categ = categ['Cuisine'].tolist()

In [5]:
df = df[df.categories.str.contains('|'.join(categ))]

In [6]:
def str_to_dict(x):
    if type(x) == float:
        return {}
    else:
        return ast.literal_eval(x)

In [7]:
df['hours'] = df['hours'].apply(str_to_dict)
df = pd.concat([df.drop(['hours'], axis=1), df['hours'].apply(pd.Series)], axis=1)

In [8]:
temp = df.set_index('business_id').categories.str.split(', ', expand=True).stack()
temp = pd.get_dummies(temp).groupby(level=0).sum()
temp = temp.loc[:, (temp.sum() >= 500)]

In [9]:
result = pd.merge(df, temp, left_on='business_id', right_index=True,
                  how='inner', sort=False)
result = result.drop(['categories'], axis = 1)

In [10]:
result['Music'] = result['Music'].apply(str_to_dict)
result = pd.concat([result.drop(['Music'], axis=1), result['Music'].apply(pd.Series)], axis=1)

In [11]:
def musicfunc (row):
    if row['dj'] == True or row['background_music'] == True or row['jukebox'] == True or row['live'] == True or row['video'] == True or row['karaoke'] == True:
        return True
    else:
        return False
result['Anymusic'] = result.apply(lambda row: musicfunc(row), axis = 1)    
   

In [12]:
result['Anymusic'].value_counts()

False    49689
True      1487
Name: Anymusic, dtype: int64

In [13]:
result = result.loc[:, (result.isnull().sum(axis=0) <= 47000)]

In [14]:
import datetime
result[['Opening_Mon','Closing_Mon']] = result['Monday'].str.split('-',expand=True)
result[['Opening_Tue','Closing_Tue']] = result['Tuesday'].str.split('-',expand=True)
result[['Opening_Wed','Closing_Wed']] = result['Wednesday'].str.split('-',expand=True)
result[['Opening_Thu','Closing_Thu']] = result['Thursday'].str.split('-',expand=True)
result[['Opening_Fri','Closing_Fri']] = result['Friday'].str.split('-',expand=True)
result[['Opening_Sat','Closing_Sat']] = result['Saturday'].str.split('-',expand=True)
result[['Opening_Sun','Closing_Sun']] = result['Sunday'].str.split('-',expand=True)


In [15]:
for i in [col for col in result if col.startswith('Opening') or col.startswith('Closing')]:
    result[i] = pd.to_datetime(result[i], format = '%H:%M')

In [16]:
def working_hours(name, clhrs, ophrs):
    result['{}'.format(name)] = np.nan
    for index,row in result.iterrows():
        value = pd.Timedelta(row['{}'.format(clhrs)] - row['{}'.format(ophrs)]).seconds / 3600
        #datetime.datetime.combine(datetime.date.min, row['Closing_Wed']) - datetime.datetime.combine(datetime.date.min,row['Opening_Wed'] )
        result.set_value(index,'{}'.format(name),value)

In [ ]:
working_hours('WD_Mon', 'Closing_Mon', 'Opening_Mon')
working_hours('WD_Tue', 'Closing_Tue', 'Opening_Tue')
working_hours('WD_Wed', 'Closing_Wed', 'Opening_Wed')
working_hours('WD_Thu', 'Closing_Thu', 'Opening_Thu')
working_hours('WD_Fri', 'Closing_Fri', 'Opening_Fri')
working_hours('WD_Sat', 'Closing_Sat', 'Opening_Sat')
working_hours('WD_Sun', 'Closing_Sun', 'Opening_Sun')

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [ ]:
for i in [col for col in result if col.startswith('Opening') or col.startswith('Closing')]:
    result[i] = result[i].dt.time

In [ ]:
result = result.drop(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], axis = 1)
result = result[result.columns.drop(list(result.filter(regex='Closing')))]

In [ ]:
def quot_cols(df, col_name):
    return df[col_name].astype(str).map(lambda x: re.findall(r"'(.*?)'", x)).str[0]

In [ ]:
columns_quotes = ['Alcohol','WiFi','NoiseLevel', 'RestaurantsAttire']
for i in columns_quotes:
    result[i] = quot_cols(result,i)

In [ ]:
result['BusinessParking'] = result['BusinessParking'].apply(str_to_dict)
result = pd.concat([result.drop(['BusinessParking'], axis=1), result['BusinessParking'].apply(pd.Series)], axis=1)
result['GoodForMeal'] = result['GoodForMeal'].apply(str_to_dict)
result = pd.concat([result.drop(['GoodForMeal'], axis=1), result['GoodForMeal'].apply(pd.Series)], axis=1)

In [ ]:
result['Ambience'] = result['Ambience'].apply(str_to_dict)
result = pd.concat([result.drop(['Ambience'], axis=1), result['Ambience'].apply(pd.Series)], axis=1)

In [ ]:
result2 = result.loc[:, (result.isnull().sum(axis=0) <= 2000)] # drop columns with more than 2000 NAs

In [ ]:
result2.shape

In [ ]:
result2 = result2.dropna()

In [ ]:
result2 = result2.drop(['Restaurants'], axis = 1)

In [ ]:
result2 = result2.rename(columns={'RestaurantsPriceRange2':'Price'})

## New features

Adding name length as feature

In [ ]:
result2['name_length']  = result2['name'].str.len().astype(int)
result2['stars']=result2['stars'].astype(float)
result2.Price = pd.to_numeric(result2.Price, errors='coerce')
result2['review_count'] = result2['review_count'].astype(float)

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
result2['review_count'] = min_max_scaler.fit_transform(result2['review_count'].values.reshape(-1,1))

## Save the clean data

In [ ]:
result2.to_csv('../data/data_clean_new.csv')

Back to the <a href="http://localhost:8888/notebooks/Documents/GitHub/DMML2019_Team_Tissot/code/Main.ipynb#EDA">Main Notebook</a>